In [1]:
import os, sys
import pandas as pd
import numpy as np
import glob

In [ ]:
# in google colab
from google.colab import drive
drive.mount('/content/drive')

# concat files

In [ ]:
# defines path to csv files
path = "/content/drive/MyDrive/ColabNotebooks/RAW/"
os.chdir(path)
dirs = os.listdir(path)

# add '城市代碼'
# select filename as '城市代碼'
all_filenames = []
city_codes = []
for file in glob.glob('*/*lvr_land_A.csv'):
  city_code = file[6]
  all_filenames.append(file)
  city_codes.append(city_code)

# check
print(all_filenames)
print(city_codes)

# add '城市代碼' to each file and write to new csv
for each_df, code in zip(all_filenames, city_codes):
  for i in range(0,len(each_df)):
      try:
        each_df = pd.read_csv(each_df)
        each_df['城市代碼'] = code
        each_df.to_csv('combined_A.csv', encoding='utf-8-sig', mode='a')
      except:
         continue


# df = pd.read_csv('combined_A.csv', each_df,error_bad_lines=False)
# print(df)

# preprocessind

In [2]:
# creat a dict for replace floors
replace_dict = {
    "地下": -1, 
    "全" : 0,
    "一層" : 1, 
    "二層" : 2, 
    "三層" : 3, 
    "四層" : 4, 
    "五層" : 5, 
    "六層" : 6, 
    "七層" : 7, 
    "八層" : 8, 
    "九層" : 9, 
    "十層" : 10, 
    "十一層" : 11, 
    "十二層" : 12, 
    "十三層" : 13, 
    "十四層" : 14, 
    "十五層" : 15, 
    "十六層" : 16, 
    "十七層" : 17, 
    "十八層" : 18, 
    "十九層" : 19, 
    "二十層" : 20,
    }

In [3]:
df = pd.read_csv('combined_A.csv',on_bad_lines='skip')
print(df.columns)
print(df)

C:\Users\Student\AppData\Local\Temp\ipykernel_18956\1804091264.py:1: DtypeWarning: Columns (4,8,15,16,17,18,19,22,23,25,26,29,30,31,32,33) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('combined_A.csv',on_bad_lines='skip')


Index(['Unnamed: 0', '鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別',
       '車位移轉總面積(平方公尺)', '車位總價元', '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯',
       '移轉編號', '城市代碼'],
      dtype='object')
         Unnamed: 0                                   鄉鎮市區              交易標的  \
0               0.0  The villages and towns urban district  transaction sign   
1               1.0                                    烏日區                土地   
2               2.0                                    烏日區                建物   
3               3.0                                    清水區         房地(土地+建物)   
4               4.0                                    新社區         房地(土地+建物)   
...             ...                                    ...               ...   
3416543       361.0      

In [4]:
df = df[df['交易標的'].notnull()]
df.loc[df['交易標的'].str.contains('建物'),:]
# df = df[df['備註'].isnull()]
df_mask=df['主要用途']=='住家用'
df = df[df_mask]

df['土地移轉總面積平方公尺'] = pd.to_numeric(df['土地移轉總面積平方公尺'], errors='coerce')
df['建物移轉總面積平方公尺'] = pd.to_numeric(df['建物移轉總面積平方公尺'], errors='coerce')
df['建物現況格局-房'] = pd.to_numeric(df['建物現況格局-房'], errors='coerce')
df['建物現況格局-廳'] = pd.to_numeric(df['建物現況格局-廳'], errors='coerce')
df['建物現況格局-衛'] = pd.to_numeric(df['建物現況格局-衛'], errors='coerce')
df['建物現況格局-隔間'] = pd.to_numeric(df['建物現況格局-隔間'], errors='coerce')
df['總價元'] = pd.to_numeric(df['總價元'], errors='coerce')
df['單價元平方公尺'] = pd.to_numeric(df['單價元平方公尺'], errors='coerce')
df['車位移轉總面積(平方公尺)'] = pd.to_numeric(df['車位移轉總面積(平方公尺)'], errors='coerce')
df['車位總價元'] = pd.to_numeric(df['車位總價元'], errors='coerce')
df['主建物面積'] = pd.to_numeric(df['主建物面積'], errors='coerce')
df['附屬建物面積'] = pd.to_numeric(df['附屬建物面積'], errors='coerce')
df['陽台面積'] = pd.to_numeric(df['陽台面積'], errors='coerce')
df['交易年月日'] = pd.to_numeric(df['交易年月日'], errors='coerce')
df['建築完成年月'] = pd.to_numeric(df['建築完成年月'], errors='coerce')


df['土地使用分區'] = df['都市土地使用分區'] +  ' ' + df['非都市土地使用分區']

df['移轉層次'] = df['移轉層次'].replace(replace_dict, regex=True)

df['鄉鎮代碼'] = df['城市代碼'] + df['鄉鎮市區']


df['交易年月日'] = df['交易年月日'].astype(str)
df['交易年度'] = df['交易年月日'].str[-8:-4].astype(str)
df['交易月度'] = df['交易年月日'].str[-8:-2].astype(str)
# df['交易年度'] = pd.to_numeric(df['交易年月日'], errors='coerce') + 1911
# df['交易月度'] = pd.to_numeric(df['交易年月日'], errors='coerce') + 191100

df['交易年_temp'] = df['交易年月日'][-7:-5]
df['建築年_temp'] = df['建築完成年月'][-5:-3]
df['交易年_temp'] = pd.to_numeric(df['交易年_temp'], errors='coerce')
df['建築年_temp'] = pd.to_numeric(df['建築年_temp'], errors='coerce')
df['屋齡'] = df['交易年_temp'] - df['建築年_temp']

df['交易筆棟數'] = df['交易筆棟數'].str[:-4].astype(str)

# df = df.drop(['Unnamed: 0','備註','土地位置建物門牌','都市土地使用分區','非都市土地使用分區','非都市土地使用編定','總樓層數','編號','移轉編號','鄉鎮市區','建築完成年月','交易年_temp','建築年_temp'], axis=1)

df.to_csv( "combined_A_temp.csv", index=False, encoding='utf-8-sig')

In [5]:
print(df.columns)
print(df)

Index(['Unnamed: 0', '鄉鎮市區', '交易標的', '土地位置建物門牌', '土地移轉總面積平方公尺', '都市土地使用分區',
       '非都市土地使用分區', '非都市土地使用編定', '交易年月日', '交易筆棟數', '移轉層次', '總樓層數', '建物型態',
       '主要用途', '主要建材', '建築完成年月', '建物移轉總面積平方公尺', '建物現況格局-房', '建物現況格局-廳',
       '建物現況格局-衛', '建物現況格局-隔間', '有無管理組織', '總價元', '單價元平方公尺', '車位類別',
       '車位移轉總面積(平方公尺)', '車位總價元', '備註', '編號', '主建物面積', '附屬建物面積', '陽台面積', '電梯',
       '移轉編號', '城市代碼', '土地使用分區', '鄉鎮代碼', '交易年度', '交易月度', '交易年_temp',
       '建築年_temp', '屋齡'],
      dtype='object')
         Unnamed: 0 鄉鎮市區          交易標的              土地位置建物門牌  土地移轉總面積平方公尺  \
2               2.0  烏日區            建物        台中市烏日區三榮路二段６７號         0.00   
3               3.0  清水區     房地(土地+建物)      台中市清水區和睦路５０巷１３１號       155.50   
6               6.0  潭子區     房地(土地+建物)  台中市潭子區勝利十二街８７巷 ２２號五樓        12.04   
7               7.0  北屯區     房地(土地+建物)  台中市北屯區天津路四段３０３號１３樓之７         1.88   
8               8.0  北屯區  房地(土地+建物)+車位     台中市北屯區崇德路二段468號4樓        14.43   
...             ...  ...           ...              

In [ ]:
df = pd.get_dummies(data=df, columns=['土地使用分區','建物型態','主要建材','有無管理組織','車位類別','電梯','城市代碼','鄉鎮代碼','交易年度','交易月度'], dummy_na=True)
df.to_csv( "combined_A_temp_getdummies.csv", index=False, encoding='utf-8-sig')